### Importamos librerias

In [21]:
import pandas
import numpy
import random
import pygame
from collections import deque

### Definimos variables globales

In [22]:
ancho=1000 #Ancho de la Pantalla
alto=1000  #Largo de la Pantalla
fps= 22 #Fps a los que se mueve el juego
vel = .2 #Velocidad de la serpiente recomendacion dejarlo en divisores de 1 
celdas=23   #Celdas del tablero recomendado para ver 23*23 celdas
thickness = 5 #Circunferencia del cuerpo


tablero=[] #Matriz del tablero donde |-1 Casilla sin nada |-100 Casilla de borde |-50 Cuerpo de la serpiente |100 Comida de la serpiente
sitio=[0,0] #Sitio actuual de Snake en forma x,y
newSitio=[0,0] #Sitio a donde va a ir Snake en forma x,y
comida=[0,0] #Sitio actual de la comida en forma x,y
fila = deque() # Crear una cola vacía para saber donde esta el cuerpo de snake


comidaExist = False # Booleano si la comida existe
moviendome = False #Actualmente se esta moviendo?
gameover= False
start=True

indexdireccion=3
direccion=["der","izq","aba", "arr"] #Arreglo de las direcciones a 3letras para comodidad
aDonde="" #Direccion actual de snake

### Funciones del juego

In [23]:
def crearSnake():
    global tablero, sitio, fila # Hacemos variables globales para modificar su valor dentro
    sitio[0] = random.randint(1, celdas) # X del sitio donde va a estar la cabeza de snake
    sitio[1] = random.randint(1, celdas) # Y del sitio donde va a estar la cabeza de snake
    tablero[sitio[1]][sitio[0]]=-50 
    
def creaTablero():
    global tablero, sitio, fila # Hacemos variables globales para modificar su valor dentro
    
    for i in range( celdas + 2): #Hacemos una matriz de -1 con borde -100 O(n²)
        temp = []
        if i == 0 or i == celdas + 1:
            for j in range(celdas + 2):
                temp.append( -100 ) 
        else:
            for j in range(celdas + 2):
                if j==0 or j==celdas+1:
                    temp.append( -100 )
                else:
                    temp.append( -1 )
        tablero.append( temp )
        
    crearSnake() #Llamamos a la funcion para crear a snake
    
    fila.append([sitio[0],sitio[1]]) # Ponemos la posicion de snake en la fila para saber donde esta la cabeza
    return 

def creaComida():
    global comidaExist, comida
    while True: #Repetimos el loop hasta que se pueda crear la comida
        numero_aleatoriox = random.randint(1, celdas) #X
        numero_aleatorioy = random.randint(1, celdas) #Y
        if tablero[numero_aleatorioy][numero_aleatoriox]==-1 : #Verificamos q no exista snake en la possicion generada
            tablero[numero_aleatorioy][numero_aleatoriox]=100 #Creamos la comida en la matrizw
            comida[0]=numero_aleatoriox #Guardamos la posicion de X en la comida
            comida[1]=numero_aleatorioy #Guardamos la posicion de Y en la comida
            comidaExist = True
            break

def dibujaCeldas(screen): # Dibujamos las celdas
    color = "black"
    for i in range(celdas):
        temporal=i*alto/celdas
        start_pos = (0, temporal)
        end_pos = (ancho, temporal)
        pygame.draw.line(screen, color, start_pos, end_pos, thickness)

        temporal=i*ancho/celdas
        start_pos = (temporal, 0)
        end_pos = (temporal, alto)
        pygame.draw.line(screen, color, start_pos, end_pos, thickness)
    
def dibujaSnakeyComida(screen): 
    global tablero # Llamamos la variables que se modifica
    mat=[]
    for i in range(celdas+2):
        temp=[]
        if i==0 or i==celdas+1:
            for j in range(celdas+2):
                temp.append(-100) 
        else:
            for j in range(celdas+2):
                if j==0 or j==celdas+1:
                    temp.append(-100)
                else:
                    temp.append(-1)
        mat.append(temp) #LLenamos la matriz
   
    cabeza=fila[-1] #Guardamos las cordenadas de la cabeza
    for i in fila: # Pintamos a toda snake de verde
        mat[i[1]][i[0]]=-50
        pygame.draw.circle(screen, "green",( ancho/celdas*(i[0]-1) + ancho/celdas/2 , alto/celdas*(i[1]-1) + alto/celdas/2), 20)
            
    mat[cabeza[1]][cabeza[0]]=0 # Pintamos la cabeza de snake de negro
    pygame.draw.circle(screen, "black",( ancho/celdas*(cabeza[0]-1) + ancho/celdas/2 , alto/celdas*(cabeza[1]-1) + alto/celdas/2), 20)
            
    mat[comida[1]][comida[0]]=100 # Pintamos la comida
    pygame.draw.circle(screen, "red",( ancho/celdas*(comida[0]-1) + ancho/celdas/2 , alto/celdas*(comida[1]-1) + alto/celdas/2), 20)
    tablero=mat

def dibuja(screen):
    dibujaSnakeyComida(screen) # Dibujamos a snake y Comidaw
    if not (comidaExist): #Verificamos la existencia de la comida si no existe:
        creaComida() #Creamos la comida
    dibujaCeldas(screen) # Dibujamos celdas
    dibujaSnakeyComida(screen) # Dibujamos a snake y Comida
    
def move(): 
    global sitio, newSitio, aDonde, comidaExist, fila, moviendome, gameover, moviendomen #Variables a modificar y usar
    x = sitio[0] #renombramos para mayor comodidad
    y = sitio[1]
    nx =newSitio[0]
    ny =newSitio[1]


    if not moviendome: # Si no hay movimiento asignamos una direccion
        if direccion[indexdireccion] == "der" and (aDonde!="izq" or len(fila)<2) : # Hcemos que no se salga ni choque en si misma de momento
            aDonde="der"
            moviendome= True
            nx= x+1

        elif direccion[indexdireccion] == "izq" and (aDonde!="der" or len(fila)<2) : # Hcemos que no se salga ni choque en si misma de momento
            aDonde="izq"
            moviendome= True
            nx= x-1

        elif direccion[indexdireccion] == "aba" and (aDonde!="arr" or len(fila)<2)  : # Hcemos que no se salga ni choque en si misma de momento
            aDonde="aba"
            moviendome= True
            ny= y+1

        elif direccion[indexdireccion] =="arr" and (aDonde!="aba" or len(fila)<2)  : # Hcemos que no se salga ni choque en si misma de momento
            aDonde="arr"
            moviendome= True
            ny= y-1 
        else:
            moviendome= True
            if  aDonde=="izq":
                nx= x-1
            if  aDonde=="der":
                nx= x+1
            if  aDonde=="arr":
                ny= y-1 
            if  aDonde=="aba":
                ny= y+1 
                
    else: # Nos movemos de acuerdo a la direccion
        if aDonde == "der":
            x=x+vel

        if aDonde == "izq":
            x=x-vel
        
        if aDonde == "aba":
            y=y+vel
        
        if aDonde == "arr":
            y=y-vel
            
            
    if aDonde == "der": 
        if (x>nx): # Cuando llegamos  decimos que ya no nos movemos 
            moviendome=False
            x=nx

            

            if tablero[y][x]==100:
                comidaExist=False
                fila.append([x,y])
            elif tablero[y][x]==-100: # si es comida agregamos a snake una bolita mas
                    gameover=True
            elif tablero[y][x]==-50: # si es comida agregamos a snake una bolita mas
                    gameover=True
            else: # Sino agregamos la bolita de posicion y quitamos la cola
                fila.append([x,y])
                fila.popleft()
            
            
            

            

    if aDonde == "izq": 
        if (x<nx): # Cuando llegamos  decimos que ya no nos movemos 
            moviendome=False
            x=nx
            if tablero[y][x]==100:
                comidaExist=False
                fila.append([x,y])
            elif tablero[y][x]==-100: # si es comida agregamos a snake una bolita mas
                    gameover=True
            elif tablero[y][x]==-50: # si es comida agregamos a snake una bolita mas
                    gameover=True
            else: # Sino agregamos la bolita de posicion y quitamos la cola
                fila.append([x,y])
                fila.popleft()

    
    if aDonde == "aba": 
        if (y>ny): # Cuando llegamos  decimos que ya no nos movemos 
            moviendome=False
            y=ny
            if tablero[y][x]==100: # si es comida agregamos a snake una bolita mas
                comidaExist=False
                fila.append([x,y])
            elif tablero[y][x]==-100: # si es comida agregamos a snake una bolita mas
                    gameover=True
            elif tablero[y][x]==-50: # si es comida agregamos a snake una bolita mas
                    gameover=True
            else: # Sino agregamos la bolita de posicion y quitamos la cola
                fila.append([x,y])
                fila.popleft()
 

    if aDonde == "arr":
        if (y<ny):
            moviendome=False
            y=ny
            if tablero[y][x]==100: # si es comida agregamos a snake una bolita mas
                comidaExist=False
                fila.append([x,y])
            elif tablero[y][x]==-100: # si es comida agregamos a snake una bolita mas
                    gameover=True
            elif tablero[y][x]==-50: # si es comida agregamos a snake una bolita mas
                    gameover=True
            else: # Sino agregamos la bolita de posicion y quitamos la cola
                fila.append([x,y])
                fila.popleft()

    sitio[0] = x 
    sitio[1] = y
    newSitio[0] = nx 
    newSitio[1] = ny


In [24]:
def init():
    global indexdireccion, start
    creaTablero()
    pygame.init()
    screen = pygame.display.set_mode((ancho, alto))
    clock = pygame.time.Clock()
    running = True
    while running:
    # poll for events
    # pygame.QUIT event means the user clicked X to close your window
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                start =False
                return
        keys = pygame.key.get_pressed()
        if keys[pygame.K_w]:
            indexdireccion=3
        if keys[pygame.K_a]:
            indexdireccion=1
        if keys[pygame.K_s]:
            indexdireccion=2
        if keys[pygame.K_d]:
            indexdireccion=0

        # fill the screen with a color to wipe away anything from last frame
        screen.fill((255, 253, 208))

        # RENDER YOUR GAME HERE
        dibuja(screen)
        move()

        

        if gameover:
            return
        
        # Definir el grosor de la línea (en este caso, 5 píxeles) 

        # Dibujar la línea en la ventana

        # flip() the display to put your work on screen
        pygame.display.flip()

        clock.tick(fps)  # limits FPS to 60

    pygame.quit()

In [25]:
while start:
    tablero=[] #Matriz del tablero donde |-1 Casilla sin nada |-100 Casilla de borde |-50 Cuerpo de la serpiente |100 Comida de la serpiente
    sitio=[0,0] #Sitio actuual de Snake en forma x,y
    newSitio=[0,0] #Sitio a donde va a ir Snake en forma x,y
    comida=[0,0] #Sitio actual de la comida en forma x,y
    fila = deque() # Crear una cola vacía para saber donde esta el cuerpo de snake


    comidaExist = False # Booleano si la comida existe
    moviendome = False #Actualmente se esta moviendo?
    gameover= False
    

    indexdireccion=3
    direccion=["der","izq","aba", "arr"] #Arreglo de las direcciones a 3letras para comodidad
    aDonde="" #Direccion actual de snake
    gameover= False
    init()
    pygame.quit()

1
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
[-100, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -100]
[-100, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -100]
[-100, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -100]
[-100, -1, -1, 100, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -100]
[-100, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -100]
[-100, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -100]
[-100, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -100]
[-100, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -100]
[-100,